In [ ]:
%pip uninstall --y tensorflow

Found existing installation: tensorflow 2.15.0
Uninstalling tensorflow-2.15.0:
  Successfully uninstalled tensorflow-2.15.0


In [ ]:
%%capture
%pip install transformers[torch]
%pip install datasets
%pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00


In [ ]:
%pip install huggingface_hub

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset, load_dataset, DatasetDict
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
import requests
import pandas as pd
import numpy as np
import evaluate


# label2id & id2label
idTolabel = {0: "fake", 1: "real"}

# Load tokenizer and model
model_name = "airesearch/wangchanberta-base-att-spm-uncased"
# model_name = "yartyjung/model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2,id2label=idTolabel)

# Dataset urls variable
train_url = "https://raw.githubusercontent.com/Yartyjung/AI-Builders-2024/main/dataset/train.csv"
validation_url = "https://raw.githubusercontent.com/Yartyjung/AI-Builders-2024/main/dataset/validation.csv"
test_url = "https://raw.githubusercontent.com/Yartyjung/AI-Builders-2024/main/dataset/test.csv"

# Load dataset
train_df = pd.read_csv(train_url,sep=",")
eval_df = pd.read_csv(validation_url,sep=",")
test_df = pd.read_csv(test_url,sep=",")
label2id = {"fake":0,"real":1}

#label to id func
def labelToid(dataframe1,dataframe2,dataframe3):
  dataframe1['label'] = dataframe1['label'].map(label2id)
  dataframe2['label'] = dataframe2['label'].map(label2id)
  dataframe3['label'] = dataframe3['label'].map(label2id)

labelToid(train_df,eval_df,test_df)

# Tokenize datasets
def tokenize_function(dataset_name):
    return tokenizer(dataset_name['text'], padding=True,truncation=True)

dataset_dict = DatasetDict({
    "train":Dataset.from_pandas(train_df),
    "eval":Dataset.from_pandas(eval_df),
    "test":Dataset.from_pandas(test_df)
})
# dataset_dict["train"][:10]
train_dataset = dataset_dict['train'].map(tokenize_function)
eval_dataset = dataset_dict['eval'].map(tokenize_function)
test_dataset = dataset_dict['test'].map(tokenize_function)

# Prepare data loaders
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16, collate_fn=data_collator)
eval_dataloader = DataLoader(eval_dataset, batch_size=16, collate_fn=data_collator)
test_dataloader = DataLoader(test_dataset, batch_size=16, collate_fn=data_collator)

#metric
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    clf_metrics = evaluate.combine(["accuracy", "f1", "precision","recall"])
    return clf_metrics.compute(predictions, labels)
# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=30,
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end = True,
    metric_for_best_model ="accuracy"

)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train
trainer.train()

model.save_pretrained('./trained_model')
tokenizer.save_pretrained('./trained_model')

model.push_to_hub("model")
tokenizer.push_to_hub("model")

tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/905k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/423M [00:00<?, ?B/s]

...

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1325 [00:00<?, ? examples/s]

Map:   0%|          | 0/1326 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.476100,0.509958,0.732075,0.732479,0.618321,0.898336
2,0.445700,0.498773,0.742642,0.698497,0.669492,0.730129
3,0.562600,0.546174,0.704151,0.709199,0.592317,0.883549
4,0.491000,0.684633,0.731321,0.745351,0.607935,0.963031
5,0.642300,0.515666,0.681509,0.715633,0.563097,0.981516
6,0.416300,0.505949,0.723774,0.738944,0.601626,0.957486
7,0.486300,0.469388,0.777358,0.769351,0.666667,0.909427
8,0.451400,0.475231,0.773585,0.772382,0.655084,0.940850
9,0.429200,0.468000,0.799245,0.759494,0.743363,0.776340
10,0.387300,0.494769,0.798491,0.758808,0.742049,0.776340


KeyboardInterrupt: 

#push epoch 9 to huggingface model

In [ ]:
modelname = "/content/results/checkpoint-2241"
tokenizer = AutoTokenizer.from_pretrained(modelname)
model = AutoModelForSequenceClassification.from_pretrained(modelname, num_labels=2,id2label=idTolabel)

model.push_to_hub("model")
tokenizer.push_to_hub("model")


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/421M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/yartyjung/model/commit/44650d53545cd9bc855733191a8613db09cf70d2', commit_message='Upload tokenizer', commit_description='', oid='44650d53545cd9bc855733191a8613db09cf70d2', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset, load_dataset, DatasetDict
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
import requests
import pandas as pd
import numpy as np
import evaluate


# label2id & id2label
idTolabel = {0: "fake", 1: "real"}

# Load tokenizer and model
# model_name = "airesearch/wangchanberta-base-att-spm-uncased"
model_name = "yartyjung/model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2,id2label=idTolabel)

# Dataset urls variable
train_url = "https://raw.githubusercontent.com/Yartyjung/AI-Builders-2024/main/dataset/train.csv"
validation_url = "https://raw.githubusercontent.com/Yartyjung/AI-Builders-2024/main/dataset/validation.csv"
test_url = "https://raw.githubusercontent.com/Yartyjung/AI-Builders-2024/main/dataset/test.csv"

# Load dataset
train_df = pd.read_csv(train_url,sep=",")
eval_df = pd.read_csv(validation_url,sep=",")
test_df = pd.read_csv(test_url,sep=",")
label2id = {"fake":0,"real":1}

#label to id func
def labelToid(dataframe1,dataframe2,dataframe3):
  dataframe1['label'] = dataframe1['label'].map(label2id)
  dataframe2['label'] = dataframe2['label'].map(label2id)
  dataframe3['label'] = dataframe3['label'].map(label2id)

labelToid(train_df,eval_df,test_df)

# Tokenize datasets
def tokenize_function(dataset_name):
    return tokenizer(dataset_name['text'], padding=True,truncation=True)

dataset_dict = DatasetDict({
    "train":Dataset.from_pandas(train_df),
    "eval":Dataset.from_pandas(eval_df),
    "test":Dataset.from_pandas(test_df)
})
# dataset_dict["train"][:10]
train_dataset = dataset_dict['train'].map(tokenize_function)
eval_dataset = dataset_dict['eval'].map(tokenize_function)
test_dataset = dataset_dict['test'].map(tokenize_function)

# Prepare data loaders
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16, collate_fn=data_collator)
eval_dataloader = DataLoader(eval_dataset, batch_size=16, collate_fn=data_collator)
test_dataloader = DataLoader(test_dataset, batch_size=16, collate_fn=data_collator)

#metric
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    clf_metrics = evaluate.combine(["accuracy", "f1", "precision","recall"])
    return clf_metrics.compute(predictions, labels)
# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    logging_dir='./logs',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,

)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

results = trainer.evaluate(eval_dataset=test_dataset)
print(results)

Map:   0%|          | 0/3975 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1325 [00:00<?, ? examples/s]

Map:   0%|          | 0/1326 [00:00<?, ? examples/s]